In [17]:
import os
import base64
import re
import csv
from dotenv import load_dotenv
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import *
from azure.search.documents.models import VectorizedQuery
import openai
from openai import AzureOpenAI

In [19]:
load_dotenv()

# Get Environment settings from .env file
load_dotenv()

# Azure AI Search Index Settings
service_endpoint = f"{os.getenv('AZURE_SEARCH_SERVICE_ENDPOINT')}"
index_creds = AzureKeyCredential(os.getenv("AZURE_SEARCH_INDEX_KEY"))
index_name = "images-search-with-keywords"

## Create a client for querying the index
search_client = SearchClient(endpoint=service_endpoint, index_name=index_name, credential=index_creds)
## Create an index
index_client = SearchIndexClient(service_endpoint, index_creds)

# Azure Openai Settings
openai.api_type = "azure"
openai.api_key = os.getenv("OPENAI_API_KEY")
openai.azure_endpoint = os.getenv("OPENAI_API_ENDPOINT")
openai.api_version = os.getenv("OPENAI_API_VERSION")

azure_openai_client = AzureOpenAI(
    api_key = os.getenv("OPENAI_API_KEY"),
    api_version = os.getenv("OPENAI_API_VERSION"),
    azure_endpoint = os.getenv("OPENAI_API_ENDPOINT")
)

In [20]:
def get_embedding(text, model="textembedding"): # model=[Deployment Name], DONOT change this
   text = text.replace("\n", " ")
   return azure_openai_client.embeddings.create(input = [text], model=model).data[0].embedding

sections = []
with open('outputupdated.csv', 'rt', newline='', encoding='utf-8', errors='ignore') as csvfile:
    csvreader = csv.reader(csvfile)
    item_num = 0
    for item in csvreader:
        section = {
            "id": f"{item_num}",
            "Image_name": item[0],
            "Image_path": item[1],
            "Response1": item[2],
            "Response2": item[3],
            "Caption": item[4],
            "Embedding": get_embedding(''.join(item[0:5]))
        }
        item_num += 1
        sections.append(section)
print(f"Finished Indexing: {len(sections)} items in total")


Finished Indexing: 71 items in total


In [21]:
index = SearchIndex(
    name=index_name,
    fields=[
        SimpleField(name="id", type="Edm.String", key=True),
        SearchableField(name="Image_name", type="Edm.String", analyzer_name="standard.lucene", 
                        filterable=True, sortable=True, facetable=True, searchable=True),
        SearchableField(name="Image_path", type="Edm.String", analyzer_name="standard.lucene",
                        filterable=True, sortable=True, facetable=True, searchable=True),
        SearchableField(name="Response1", type="Edm.String", analyzer_name="standard.lucene",
                        filterable=True, sortable=True, facetable=True, searchable=True),
        SearchableField(name="Response2", type="Edm.String", analyzer_name="standard.lucene",
                        filterable=True, sortable=True, facetable=True, searchable=True),
        SearchableField(name="Caption", type="Edm.String", analyzer_name="standard.lucene",
                        filterable=True, sortable=True, facetable=True, searchable=True),
        SearchField(name="Embedding", type=SearchFieldDataType.Collection(SearchFieldDataType.Single),  
            hidden=False, searchable=True, filterable=False, sortable=False, facetable=False,
            vector_search_dimensions=1536, vector_search_profile_name="my-vector-config"),
    ],
    vector_search=VectorSearch(
        profiles=[VectorSearchProfile(
            name="my-vector-config",
            algorithm_configuration_name="my-hnsw")
        ],
        algorithms=[
            HnswAlgorithmConfiguration(name="my-hnsw")
        ]
    )
)

index_client.create_or_update_index(index)

In [23]:
results = search_client.upload_documents(documents=sections)
print("Uploading")
succeeded = sum([1 for r in results if r.succeeded])
print(f"Indexed {len(results)} sections, {succeeded} succeeded")
batch = []

Uploading
Indexed 71 sections, 71 succeeded


In [29]:
image_search_query = """
Notable figures involved in establishing the University of Hong Kong include Lugard, who prepared a memorandum outlining the vision for the University and appointed a Committee of Management [3-8-1]. The Committee included prominent individuals such as May, a rector of the College, Sir Paul Chater, Mody, Ho Kai, Wei Yuk, Lau Chu-pak, and Cecil Clementi. Ho Kai and Wei Yuk were responsible for forming a fund-raising sub-committee [3-8-1]. 

Another key figure was W. H. Donald, the managing director of the China Mail, who initiated the idea of establishing a University in Hong Kong [2-8-7]. 

Sun Yat-sen, a student at the College of Medicine, was also involved in the early discussions about reform and change, which eventually played a role in the establishment of the University [2-8-3]. 

The University of Hong Kong has since produced many notable graduates who have gone on to occupy important positions in various fields, both within Hong Kong and internationally [1-1-7][1-1-17]."""

'''Vector Search for Images'''
# query_vector = get_embedding(query)
# r = search_client.search(
#     search_text=None,
#     top=3,
#     vector_queries=[VectorizedQuery(
#         vector=query_vector,
#         fields="Embedding"
#     )]
# )

'''Full text search'''
r = search_client.search(
    search_text=image_search_query,
    top=3
)

search_results = []
for result in r:
    print("#########################################")
    print("Source: " + result["Image_name"])
    print("Caption" + result["Caption"])
    search_results.append("Source: " + result["Image_name"] + result["Response1"] + result["Response2"] + "; Caption: " + result["Caption"])

#########################################
Source: Group photo of staff and students of the College.jpg
CaptionStaff & students of the College in 1906. Front row: centre, Dr Ho Kai, with Dr. James Cantlie at his left and F.H. May, the Rector of the College and future Governor of Hong Kong, Second row, from left: at his right. 1st Dr. Ma Luk; 2nd Dr. Caxion To; 5th Dr. Kwan Sum Yin; 6th, S.F. Lee (later Dr. S.F.Lee) Last row, from left: 4th, Dr. Ho Ko Tsun
#########################################
Source: Sun Yat-sen with early students of the College of Medicine.jpg
CaptionEarly students of the College of Medicine: From left: 2nd, Sun Yat-sen; 3rd, Ch'en Shao-pai, 4th, Kwan Sum Yin. The other two were Yang Ho-ling & Yu Lieh, who also shared Sun's revolutionary fervour. The four (excepting Kwan) were known as the "Four Bandits" for their radical ideas.
#########################################
Source: Group photo of students of the College of Medicine with Dr Gibson.jpg
CaptionGroup phot

In [ ]:
systemMessage = """AI Assistant that helps user to answer questions from sources provided. Be brief in your answers, with supporting evidence from only the sources below.
                    Answer ONLY with the facts listed in the list of sources below. 
                    If there isn't enough information below, say you don't know. Do not generate answers that don't use the sources below. 
                    Each source has a name followed by colon and the actual information, always include the source name for each fact you use in the response. 
                    Use square brackets to reference the source, e.g. [info1.txt]. Don't combine sources, list each source separately, e.g. [info1.txt][info2.pdf].
                """

messages = [
    {'role' : 'system', 'content' : systemMessage},
    {'role' : 'user', 'content' : query + "   Source:" + " ".join(search_results)}
]

chat_completion = openai.chat.completions.create(
    model="summer", # Do not edit this. model="deployment_name"
    messages=messages, 
    temperature=0.7, 
    max_tokens=1024, 
    n=1)

chat_content = chat_completion.choices[0].message.content
print(chat_content)

The major milestones in the university's founding and early history include its founding in 1911 by Sir Frederick Lugard and others to help in a Chinese renaissance [2-1-2]. A memorandum was prepared in January 1909 by Lugard, outlining the considerations for the university project and the kind of university he expected it to be [3-8-1]. 

A significant event was Hormusjee Nowrojee Mody's change of mind about Oxford and Cambridge as models and his decision to incorporate a secondary school as part of a university institution. Mody offered HK$150,000 for the erection of the first buildings and the endowment of a University [3-3-1].

Another milestone was the drafting of an editorial by W. H. Donald, the managing director of the China Mail, which has generally been accepted as the origin point of the notion of a University of Hong Kong [2-8-7]. The bill of incorporation was read in Legislative Council, with the incorporation becoming effective on the 23rd May 1907 [3-6-4].


In [11]:
'''Not this part on Generating Answer'''

systemMessage = """AI Assistant that helps user to answer questions from sources provided. Be brief in your answers.
                    Answer ONLY with the facts listed in the list of sources below. 
                    If there isn't enough information below, say you don't know. Do not generate answers that don't use the sources below. 
                    Each source has a name followed by colon and the actual information, always include the source name for each fact you use in the response. 
                    Use square brackets to reference the source, e.g. [info1.txt]. Don't combine sources, list each source separately, e.g. [info1.txt][info2.pdf].
                """

messages = [
    {'role' : 'system', 'content' : systemMessage},
    {'role' : 'user', 'content' : query + "   Source:" + " ".join(search_results)}
]

chat_completion = openai.chat.completions.create(
    model="summer", # Do not edit this. model="deployment_name"
    messages=messages, 
    temperature=0.7, 
    max_tokens=1024, 
    n=1)

chat_content = chat_completion.choices[0].message.content
print(chat_content)

Sir Ho Kai was a physician and barrister trained in Britain, and he was one of the founders of the Alice Memorial Hospital in Hong Kong. This hospital was opened in February 1887 with the intention of serving the poorer classes and advancing medical science among the Chinese. Sir Ho Kai envisioned the hospital serving as a training school for students who would spread medical knowledge in China. He played a significant role in the establishment of the University of Hong Kong and was part of its fundraising sub-committee. He was also preparing to sign a contract for the construction of an anatomy laboratory and museum. Although he was friendly with Dr. Kwan, a revolutionary figure, there was no suspicion of Sir Ho Kai having any revolutionary tendencies [3-4-6][3-6-5][3-8-3].

An image of Sir Ho Kai depicts him in a suit and tie or bow tie, with a medal or decoration. This black and white photograph is accompanied by a caption in English and Chinese, which describes him as the founder o